In [1]:
!pip install najaeda==0.2.1
!pip install networkx

!gdown --folder "1XcbW6bNB-ZNmr7e0msPLvrveuwAW8Kas"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.2 MB/s eta 0:00:00
Retrieving folder contents
Processing file 1eZxGAfxtDy62HutTL3Hq29feN4EeuUIU design0.v
Processing file 1PVPiRkZIhluVlfOljRn76HmWd82kFTuS design1.v
Processing file 1lZYT3MxyKFFp-DomkLP26nwv3SCeAuA- design2.v
Processing file 1reWnxg75Jn0BKrgMZsdXx7qzx43RLkLn design3.v
Processing file 1PCTgcH6DNRIWOHKlfLt1K-QlBtHEmHgO design4.v
Processing file 1oq-Nqt-au4TOJ_C3SHpcS1hQ0vnZPkf9 design5.v
Processing file 1xYG6T_vx4WbSqix3iSAGefU5f258xjEE design6.v
Processing file 1bgJqTdTruwAI9Uq637RerhRHlDLWZzto design7.v
Processing file 1NVIL6lCuQBPMXeRnSFgmzLwbkEc5VpLe design8.v
Processing file 1U_9ZS59h0cwYrmurAuUeiaQflwtq1suE design9.v
Processing file 18JvPmHKbGEUIowqRnx2CewWiHY_ra4C_ design10.v
Processing file 1WDQvZgoHR2TSsBxHFTNBT7Eaj1SiR-_J design11.v
Processing file 1a6InfV8qOwHrfUUEVDjv8NJ1vKjqviqe design12.v
Processing file 14_86aHaHBqxWMRDVKBlqbx_IFuV79YHY design13.v
Processing file 1sLYf2kJjQoxC648JPbzIVYRz73Ec0BWW de

In [2]:
%%writefile iccadPrim.py
import logging
from najaeda import naja

def constructDFF(lib):
  cell = naja.SNLDesign.createPrimitive(lib, "dff")
  naja.SNLScalarTerm.create(cell, naja.SNLTerm.Direction.Output, "Q")
  naja.SNLScalarTerm.create(cell, naja.SNLTerm.Direction.Input, "RN")
  naja.SNLScalarTerm.create(cell, naja.SNLTerm.Direction.Input, "SN")
  naja.SNLScalarTerm.create(cell, naja.SNLTerm.Direction.Input, "CK")
  naja.SNLScalarTerm.create(cell, naja.SNLTerm.Direction.Input, "D")

def load(db):
  logging.info("Loading verilog built-in + iccad primitives")
  lib = naja.NLLibrary.createPrimitives(db, "iccad")
  constructDFF(lib)

Writing iccadPrim.py


In [3]:
import collections
import sys

def initialize_features(design, trojans = [], inputDesign = False):
    # Init features for design
    features = {}
    all_insts = list(design.getInstances())

    for inst in all_insts:
        inst_name = inst.getName()

        fanin = 0
        fanout = 0
        for instTerm in inst.getInstTerms():
            direction = instTerm.getDirection()
            if (direction == naja.SNLTerm.Direction.Output):
                net = instTerm.getNet()
                for sinkTerm in net.getInstTerms():
                    sink_direction = sinkTerm.getDirection()
                    if (sink_direction == naja.SNLTerm.Direction.Input):
                        fanout += 1
            else:
                fanin += 1

        isTrojan = 0
        if inst_name in trojans:
            isTrojan = 1
        if inputDesign:
            isTrojan = -1

        features[inst_name] = {
            'fanin': fanin,
            'fanout': fanout,
            'closestInputDepth': float(sys.float_info.max),
            'closestInputName': [],
            'closestDriveFFDepth': float(sys.float_info.max),
            'closestDriveFFName': [],
            'closestOutputDepth': float(sys.float_info.max),
            'closestOutputName': [],
            'closestSinkFFDepth': float(sys.float_info.max),
            'closestSinkFFName': [],
            'ratioFan2': float(sys.float_info.max),
            'ratioFan3': float(sys.float_info.max),
            'ratioFan4': float(sys.float_info.max),
            'ratioFan5': float(sys.float_info.max),
            'inFF2': float(sys.float_info.max),
            'inFF3': float(sys.float_info.max),
            'inFF4': float(sys.float_info.max),
            'inFF5': float(sys.float_info.max),
            'outFF2': float(sys.float_info.max),
            'outFF3': float(sys.float_info.max),
            'outFF4': float(sys.float_info.max),
            'outFF5': float(sys.float_info.max),
            'inInv2': float(sys.float_info.max),
            'inInv3': float(sys.float_info.max),
            'inInv4': float(sys.float_info.max),
            'inInv5': float(sys.float_info.max),
            'outInv2': float(sys.float_info.max),
            'outInv3': float(sys.float_info.max),
            'outInv4': float(sys.float_info.max),
            'outInv5': float(sys.float_info.max),
            'label': isTrojan
        }

    print(f"Initialization complete for {len(features)} instances.")
    return features

def annotate_features_cells(design, features):
    print("--> Sequencial Cell Annotation")
    for inst in design.getInstances():
      annotate_cell(design, features, inst)

def annotate_cell(design, features, inst):
    inst_name = inst.getName()
    cellsIn = [[] for i in range(5)]
    cellsOut = [[] for i in range(5)]

    # Save cell names from up to 5 levels behind instance
    q_back = collections.deque()
    q_back.append((inst, 0))
    visited_back = {inst}
    while q_back:
        current_inst, depth = q_back.popleft()
        if depth >= 5: continue
        cellsIn[depth].append(current_inst.getModel().getName())
        for in_term in current_inst.getInstTerms():
            if in_term.getDirection() == naja.SNLTerm.Direction.Input:
                in_net = in_term.getNet()
                if not in_net: continue
                for term_on_net in in_net.getInstTerms():
                    if term_on_net.getDirection() == naja.SNLTerm.Direction.Output:
                        prev_inst = term_on_net.getInstance()
                        if prev_inst not in visited_back:
                            q_back.append((prev_inst, depth + 1))
                            visited_back.add(prev_inst)

    # Save cell names from up to 5 levels after instance
    q_forward = collections.deque()
    q_forward.append((inst, 0))
    visited_forward = {inst}
    while q_forward:
        current_inst, depth = q_forward.popleft()
        if depth >= 5: continue
        cellsOut[depth].append(current_inst.getModel().getName())
        for out_term in current_inst.getInstTerms():
            if out_term.getDirection() == naja.SNLTerm.Direction.Output:
                out_net = out_term.getNet()
                if not out_net: continue
                for term_on_net in out_net.getInstTerms():
                    if term_on_net.getDirection() == naja.SNLTerm.Direction.Input:
                        next_inst = term_on_net.getInstance()
                        if next_inst not in visited_forward:
                            q_forward.append((next_inst, depth + 1))
                            visited_forward.add(next_inst)

    # computing ratioFan
    for i in range(1, 5): # levels 2 through 5
        level_x = i + 1
        fanin_levelX_count = len(cellsIn[i])
        ratio = 0
        if features[inst_name]['fanout'] != 0:
            ratio = fanin_levelX_count / features[inst_name]['fanout']
        features[inst_name][f'ratioFan{level_x}'] = ratio

    # computing counting inv/not and dffs
    cumulative_in_ffs = 0
    cumulative_in_invs = 0
    for i in range(5):
        level_cells = cellsIn[i]
        cumulative_in_ffs += sum(1 for cell_name in level_cells if "dff" in cell_name)
        cumulative_in_invs += sum(1 for cell_name in level_cells if "not" in cell_name)

        # paper doesn't use 0 (level 1)
        if i >= 1:
            level_x = i + 1
            features[inst_name][f'inFF{level_x}'] = cumulative_in_ffs
            features[inst_name][f'inInv{level_x}'] = cumulative_in_invs

    cumulative_out_ffs = 0
    cumulative_out_invs = 0
    for i in range(5):
        level_cells = cellsOut[i]
        cumulative_out_ffs += sum(1 for cell_name in level_cells if "dff" in cell_name)
        cumulative_out_invs += sum(1 for cell_name in level_cells if "not" in cell_name)

        if i >= 1:
            level_x = i + 1
            features[inst_name][f'outFF{level_x}'] = cumulative_out_ffs
            features[inst_name][f'outInv{level_x}'] = cumulative_out_invs

def _update_path_feature(inst_features, depth_key, name_key, new_depth, new_names):
    # Check if path needs an update
    is_better = new_depth < inst_features[depth_key]
    is_same = new_depth == inst_features[depth_key]

    if is_better:
        inst_features[depth_key] = new_depth
        inst_features[name_key] = list(new_names) # copy
    elif is_same:
        for name in new_names:
            if name not in inst_features[name_key]:
                inst_features[name_key].append(name)

    return is_better or is_same

def _process_instance(inst, features, comb_depth, comb_names, ff_depth, ff_name, is_forward):
    inst_name = inst.getName()
    inst_features = features[inst_name]

    # Determine keys based on traversal direction
    comb_depth_key = 'closestInputDepth' if is_forward else 'closestOutputDepth'
    comb_name_key = 'closestInputName' if is_forward else 'closestOutputName'
    ff_depth_key = 'closestDriveFFDepth' if is_forward else 'closestSinkFFDepth'
    ff_name_key = 'closestDriveFFName' if is_forward else 'closestSinkFFName'

    # Update features and check if we should continue traversal from this instance
    comb_path_updated = _update_path_feature(inst_features, comb_depth_key, comb_name_key, comb_depth, comb_names)
    ff_path_updated = _update_path_feature(inst_features, ff_depth_key, ff_name_key, ff_depth, [ff_name] if ff_name else [])

    if not comb_path_updated and not ff_path_updated:
        return None # Prune search if no path was improved

    # Determine next state for fanout/fanin nets
    is_ff = "dff" in inst.getModel().getName()
    next_ff_name = inst_name if is_ff else ff_name
    next_ff_depth = 0 if is_ff else ff_depth

    return (inst_features[comb_name_key], next_ff_name, next_ff_depth)


def annotate_features(design, features):
    print("--> Starting Annotation.")
    # Forwards and Backwards BFS to annotate features
    q = collections.deque()
    q_back = collections.deque()

    # Keep track of the best depths seen for each net to avoid redundant processing
    net_min_depths = {} # (net, is_forward) -> min_depth

    # Initialize queues with primary inputs and outputs
    for term in design.getBitTerms():
        is_input = term.getDirection() == naja.SNLTerm.Direction.Input
        net = term.getNet()
        if not net: continue

        # State is net, depth to port, port names, ff names, ff depth
        state = (net, 0, [str(net)], None, float(sys.float_info.max))
        if is_input:
            q.append(state)
            net_min_depths[(net, True)] = 0
        else:
            q_back.append(state)
            net_min_depths[(net, False)] = 0

    # Forward propagation
    print("Starting forward propagation...")
    while q:
        current_net, pi_depth, pi_names, last_ff, depth_from_ff = q.popleft()

        for term in current_net.getInstTerms():
            inst = term.getInstance()

            # To get the next cell, check the terms that are Inputs (so continue from any other)
            if (term.getDirection() == naja.SNLTerm.Direction.Output):
                continue

            process_result = _process_instance(inst, features, pi_depth + 1, pi_names, (depth_from_ff + 1) if last_ff else float('inf'), last_ff, is_forward=True)

            if process_result:
                new_pi_names, next_ff_name, next_depth_from_ff = process_result

                # Enqueue fanout nets
                for out_term in inst.getInstTerms():
                    if (out_term.getDirection() == naja.SNLTerm.Direction.Output):
                        out_net = out_term.getNet()
                        if not out_net: continue

                        new_pi_depth = pi_depth + 1
                        if new_pi_depth < net_min_depths.get((out_net, True), float('inf')):
                            net_min_depths[(out_net, True)] = new_pi_depth
                            q.append((out_net, new_pi_depth, new_pi_names, next_ff_name, next_depth_from_ff))

    # Backward propagation
    print("Starting backwards propagation...")
    while q_back:
        current_net, po_depth, po_names, last_ff, depth_from_ff = q_back.popleft()

        for term in current_net.getInstTerms():
            inst = term.getInstance()

            # To get the next cell, check the terms that are Outputs (so continue from any other)
            if (term.getDirection() == naja.SNLTerm.Direction.Input):
                continue

            process_result = _process_instance(inst, features, po_depth + 1, po_names, (depth_from_ff + 1) if last_ff else float('inf'), last_ff, is_forward=False)

            if process_result:
                new_po_names, next_ff_name, next_depth_from_ff = process_result

                # Enqueue fanin nets
                for in_term in inst.getInstTerms():
                    if (in_term.getDirection() == naja.SNLTerm.Direction.Input):
                        in_net = in_term.getNet()
                        if not in_net: continue

                        new_po_depth = po_depth + 1
                        if new_po_depth < net_min_depths.get((in_net, False), float('inf')):
                            net_min_depths[(in_net, False)] = new_po_depth
                            q_back.append((in_net, new_po_depth, new_po_names, next_ff_name, next_depth_from_ff))

    print("--> Annotation complete.")


In [4]:
import sys

def extract_numeric_features(all_features):
    numeric_features = {}
    for instance_name, data in all_features.items():
        # Inicializa o dicionário para a instância atual
        numeric_features[instance_name] = {}

        # Lista de chaves numéricas a serem extraídas
        numeric_keys = [
            'fanin',
            'fanout',
            'closestInputDepth',
            'closestDriveFFDepth',
            'closestOutputDepth',
            'closestSinkFFDepth',
            'ratioFan2',
            'ratioFan3',
            'ratioFan4',
            'ratioFan5',
            'inFF2',
            'inFF3',
            'inFF4',
            'inFF5',
            'outFF2',
            'outFF3',
            'outFF4',
            'outFF5',
            'inInv2',
            'inInv3',
            'inInv4',
            'inInv5',
            'outInv2',
            'outInv3',
            'outInv4',
            'outInv5',
        ]



        # Itera sobre as chaves e adiciona ao novo dicionário
        for key in numeric_keys:
            # Verifica se a chave existe e se o valor é numérico antes de adicionar
            if key in data:
                # O `label` já é numérico. Para as outras, garanta que sejam
                if isinstance(data[key], (int)):
                    numeric_features[instance_name][key] = data[key]
                # Lida com casos especiais como 'inf'
                elif data[key] >= 1e100:
                    numeric_features[instance_name][key] = -1 # Ou outro valor que represente 'infinito'
    return numeric_features


In [5]:
trojan_exemples = [
                    "design0.v",
                    "design1.v",
                    "design2.v",
                    "design3.v",
                    "design4.v",
                    "design5.v",
                    "design6.v",
                    "design7.v",
                    "design8.v",
                   "design9.v",
                   "design10.v",
                   "design11.v",
                   "design12.v",
                   "design13.v",
                   "design14.v",
                   "design15.v",
                   "design16.v",
                   "design17.v",
                   "design18.v",
                   "design19.v"



                   ]
prefix_list = [ "0",  "1",  "2", "3", "4", "5", "6", "7", "8", "9", "10",
               "11", "12", "13", "14", "15", "16", "17", "18", "19" ]

unknown_file = "/content/trojan_gabarito/design6.v"  # e desse arquivo que queremos descobrir quais gates são trojan

# dicionario com o gabarito dos gates que são trojan
trojan_gates = {}




trojan_gates_list = []

for i in prefix_list:
    result_file = f"/content/trojan_gabarito/result{i}.txt"
    print(f"Processing file: {result_file}")
    with open(result_file, "r") as f:
        inside_block = False
        for line in f:
            line = line.strip()
            if line == "TROJAN_GATES":
                inside_block = True
                continue
            elif line == "END_TROJAN_GATES":
                inside_block = False
                continue
            if inside_block and line.startswith("g"):
                prefixed = f"{i}{line}"
                trojan_gates_list.append(prefixed)

# Gabarito dos gates que são trojan no arquivo que queremos descobrir
gates_unknown = []

unknown_result = f"/content/trojan_gabarito/result6.txt"

with open(unknown_result, "r") as f:
    inside_block = False
    gates_u = []   # cria a lista antes de usar
    for line in f:
        line = line.strip()

        if line == "TROJAN_GATES":
            inside_block = True
            continue
        elif line == "END_TROJAN_GATES":
            inside_block = False
            continue

        if inside_block and line.startswith("g"):
            gates_u.append(line)

gates_unknown = gates_u


Processing file: /content/trojan_gabarito/result0.txt
Processing file: /content/trojan_gabarito/result1.txt
Processing file: /content/trojan_gabarito/result2.txt
Processing file: /content/trojan_gabarito/result3.txt
Processing file: /content/trojan_gabarito/result4.txt
Processing file: /content/trojan_gabarito/result5.txt
Processing file: /content/trojan_gabarito/result6.txt
Processing file: /content/trojan_gabarito/result7.txt
Processing file: /content/trojan_gabarito/result8.txt
Processing file: /content/trojan_gabarito/result9.txt
Processing file: /content/trojan_gabarito/result10.txt
Processing file: /content/trojan_gabarito/result11.txt
Processing file: /content/trojan_gabarito/result12.txt
Processing file: /content/trojan_gabarito/result13.txt
Processing file: /content/trojan_gabarito/result14.txt
Processing file: /content/trojan_gabarito/result15.txt
Processing file: /content/trojan_gabarito/result16.txt
Processing file: /content/trojan_gabarito/result17.txt
Processing file: /co

In [ ]:
"""
for gate in trojan_gates_list:
  print(gate)
"""

'\nfor gate in trojan_gates_list:\n  print(gate)\n'

In [ ]:
#print(f"Gates unknown_file:\n {gates_unknown}")

In [6]:
import networkx as nx
import numpy as np
from collections import defaultdict
from sklearn.metrics.pairwise import euclidean_distances
from najaeda import netlist, naja

class Features_gates():
  features_gates = {}

class Vetorial_space():
  gates_vetorial = {}

def load_design(design_file, prim_file="iccadPrim.py"):
  netlist.reset()
  netlist.load_primitives_from_file(prim_file)
  top = netlist.load_verilog(design_file)
  universe = naja.NLUniverse.get()
  design = universe.getTopDesign()

  return design


features_hyperedgs = {}
hyperedge_weights  = {}


# Pego o nome do nó para definir a chave da aresta
def get_key_edg(node):
    key = node.replace("g", "e")
    return key


def vector_node(node, gates_vetorial):
    feature_keys = [
        'fanin',
        'fanout',
        'closestInputDepth',
        'closestDriveFFDepth',
        'closestOutputDepth',
        'closestSinkFFDepth',
        'ratioFan2',
        'ratioFan3',
        'ratioFan4',
        'ratioFan5',
        'inFF2',
        'inFF3',
        'inFF4',
        'inFF5',
        'outFF2',
        'outFF3',
        'outFF4',
        'outFF5',
        'inInv2',
        'inInv3',
        'inInv4',
        'inInv5',
        'outInv2',
        'outInv3',
        'outInv4',
        'outInv5',
    ]

    # Access the specific node's features from gates_vetorial
    node_data = gates_vetorial.get(node, {})

    # Create the feature vector
    vector = [node_data.get(key, 0) for key in feature_keys]
    return vector

def features_hyperedg(gates_vetorial, size_k):
    global features_hyperedgs
    global hyperedge_weights

    nodes_list = list(gates_vetorial.keys())  # Use the keys from gates_vetorial as nodes
    node_vectors = {node: vector_node(node, gates_vetorial) for node in nodes_list}

    # Calculate the distance matrix efficiently
    node_vectors_array = np.array(list(node_vectors.values()))
    distance_matrix = euclidean_distances(node_vectors_array, node_vectors_array)
    print("Euclidean distance matrix:\n", distance_matrix)

    # Calculate sigma^2
    sigma_squared = np.mean(distance_matrix)
    if sigma_squared == 0:
        sigma_squared = 1e-6  # Ensure sigma^2 is not zero

    for i, node in enumerate(nodes_list):
        key = get_key_edg(node)  # Generate the edge key

        # Find the k-nearest neighbors
        k_nearest_indices = np.argsort(distance_matrix[i])[1:size_k + 1]
        hyperedge = [node] + [nodes_list[j] for j in k_nearest_indices]
        features_hyperedgs[key] = hyperedge

        # Calculate the Gaussian weight for the hyperedge
        total_weight = 0
        for node_j_index in k_nearest_indices:
            distance = distance_matrix[i, node_j_index]
            weight_ij = np.exp(- (distance**2) / (2 * sigma_squared))
            total_weight += weight_ij

        hyperedge_weights[key] = total_weight

    return features_hyperedgs, hyperedge_weights

In [7]:

features_g = Features_gates()
features_v = Vetorial_space()

# Processa cada arquivo separadamente
for key, design_file in enumerate(trojan_exemples):

    design_file = [f"/content/trojan_gabarito/{design_file}"]
    design = load_design(design_file, prim_file="iccadPrim.py")

    if design_file:
        print(f"\ncarregando {design_file}")

    # Inicializa as features para o design atual
    features = initialize_features(design)
    annotate_features(design, features)
    annotate_features_cells(design, features)

    # Extrai as features numéricas para o design atual
    numeric_features = extract_numeric_features(features)

    # Salva as features no dicionário global, garantindo que cada arquivo tenha suas próprias chaves
    features_g.features_gates[key] = numeric_features

# Mescla as informações de todos os arquivos
for id_file, gates in features_g.features_gates.items():
    for gate_key, gate_f in gates.items():
        # Cria uma chave única para cada gate, combinando o id do arquivo e a chave do gate
        new_key = f"{id_file}{gate_key}"

        # Verifica se a chave já existe antes de adicionar
        if new_key not in features_v.gates_vetorial:
            features_v.gates_vetorial[new_key] = gate_f

# Exibe os gates salvos
for key in features_v.gates_vetorial:
    print(f"{key} -> gates: {features_v.gates_vetorial[key]}")


A saída de streaming foi truncada nas últimas 5000 linhas.
16g1216 -> gates: {'fanin': 2, 'fanout': 1, 'closestInputDepth': 10, 'closestDriveFFDepth': 9, 'closestOutputDepth': 9, 'closestSinkFFDepth': 8, 'inFF2': 0, 'inFF3': 0, 'inFF4': 0, 'inFF5': 0, 'outFF2': 0, 'outFF3': 0, 'outFF4': 0, 'outFF5': 0, 'inInv2': 0, 'inInv3': 3, 'inInv4': 3, 'inInv5': 4, 'outInv2': 0, 'outInv3': 0, 'outInv4': 0, 'outInv5': 0}
16g1217 -> gates: {'fanin': 2, 'fanout': 2, 'closestInputDepth': 6, 'closestDriveFFDepth': 5, 'closestOutputDepth': 10, 'closestSinkFFDepth': 9, 'inFF2': 0, 'inFF3': 0, 'inFF4': 0, 'inFF5': 0, 'outFF2': 0, 'outFF3': 0, 'outFF4': 0, 'outFF5': 0, 'inInv2': 0, 'inInv3': 0, 'inInv4': 0, 'inInv5': 1, 'outInv2': 0, 'outInv3': 0, 'outInv4': 0, 'outInv5': 0}
16g1218 -> gates: {'fanin': 1, 'fanout': 1, 'closestInputDepth': 6, 'closestDriveFFDepth': 5, 'closestOutputDepth': 13, 'closestSinkFFDepth': 12, 'inFF2': 0, 'inFF3': 0, 'inFF4': 0, 'inFF5': 0, 'outFF2': 0, 'outFF3': 0, 'outFF4': 0, 'o

In [ ]:
"""
for key in features_g.features_gates:
  print(f"{key} -> gates: {features_g.features_gates[key]}")

  """

'\nfor key in features_g.features_gates:\n  print(f"{key} -> gates: {features_g.features_gates[key]}")\n\n  '

In [8]:

size_k = 10

features_hyperedges, hyperedge_weights = features_hyperedg(features_v.gates_vetorial, size_k)

Euclidean distance matrix:
 [[ 0.          4.47213595  8.48528137 ...  3.74165739 20.24845673
   5.38516481]
 [ 4.47213595  0.          7.21110255 ...  5.83095189 19.74841766
   7.93725393]
 [ 8.48528137  7.21110255  0.         ...  9.2736185  18.86796226
   9.94987437]
 ...
 [ 3.74165739  5.83095189  9.2736185  ...  0.         20.39607805
   6.244998  ]
 [20.24845673 19.74841766 18.86796226 ... 20.39607805  0.
  22.64950331]
 [ 5.38516481  7.93725393  9.94987437 ...  6.244998   22.64950331
   0.        ]]


In [ ]:
"""
for hyperedge in features_hyperedges:
  print(f"{hyperedge} -> gates: {features_hyperedges[hyperedge]}")
  """

'\nfor hyperedge in features_hyperedges:\n  print(f"{hyperedge} -> gates: {features_hyperedges[hyperedge]}")\n  '

Buscando por hiperarestas

In [9]:
def find_trojan(features_hyperedges, trojan_gates_list, number_trojan_hyperedges, unknown_design):
    candidates = []
    trojan_set = set(trojan_gates_list)

    for edge_key, nodes in features_hyperedges.items():
        found_trojans = list(set(nodes) & trojan_set)
        trojan_count = len(found_trojans)

        has_unknown = any(node.startswith(unknown_design) for node in nodes)
        design_gates = [node for node in nodes if node.startswith(unknown_design)]

        if trojan_count >= number_trojan_hyperedges and has_unknown:
            number_unknown = sum(1 for node in nodes if node.startswith(unknown_design))
            info = {
                "edge_key": edge_key,
                "trojan_count": trojan_count,
                "count unknown": number_unknown,
                "found_trojans": found_trojans,
                "design_gates": design_gates,

                "hyperedge": nodes
            }
            candidates.append(info)


    return candidates


In [10]:
def print_candidates(candidates):
    if not candidates:
        print("Sem info")
        return

    for info in candidates:
        print(f"Edge Key: {info['edge_key']}")
        print(f"  count unknown: {info['count unknown']}")
        print(f"  Trojan Count: {info['trojan_count']}")
        print(f"  Found Trojans: {info['found_trojans']}")
        print(f"  Design Gates: {info['design_gates']}")
        print(f"  Hyperedge: {info['hyperedge']}")
        print("-" * 40)

In [11]:
def list_gates(candidates, unknown_design):
    gate_list = []
    count = 0

    for hyperedge in candidates:
        for node in hyperedge["hyperedge"]:
            if node.startswith(unknown_design) and node not in gate_list:
                gate_list.append(node)
                count += 1

    return gate_list, count


In [12]:
desig_s = "6g"
candidates= find_trojan(features_hyperedges, trojan_gates_list, 5, desig_s )
print_candidates(candidates)


Edge Key: 0e945
  count unknown: 1
  Trojan Count: 6
  Found Trojans: ['7g2094', '7g1047', '7g744', '6g1651', '7g1902', '7g1164']
  Design Gates: ['6g1651']
  Hyperedge: ['0g945', '2g281', '2g255', '0g945', '7g744', '7g1047', '7g1164', '6g1651', '7g2094', '7g1902', '2g132']
----------------------------------------
Edge Key: 1e448
  count unknown: 2
  Trojan Count: 6
  Found Trojans: ['14g6112', '16g714', '6g2496', '6g984', '14g3947', '9g2686']
  Design Gates: ['6g984', '6g2496']
  Hyperedge: ['1g448', '1g210', '7g1115', '14g6112', '14g3947', '9g2686', '1g209', '16g2706', '16g714', '6g984', '6g2496']
----------------------------------------
Edge Key: 1e566
  count unknown: 2
  Trojan Count: 9
  Found Trojans: ['9g2054', '9g4141', '16g167', '6g2031', '9g3383', '9g3772', '9g3817', '6g1341', '9g2070']
  Design Gates: ['6g2031', '6g1341']
  Hyperedge: ['1g566', '0g1659', '9g3817', '9g2070', '16g167', '9g4141', '9g3772', '9g3383', '6g2031', '6g1341', '9g2054']
-------------------------------

In [ ]:
def automated_trojan_detection(trojan_exemples, prefix_list, size_k, min_trojan_threshold=5):

    results_dict = {}

    # Processa cada design individualmente
    for target_idx, target_design in enumerate(trojan_exemples):
        target_prefix = str(target_idx)
        print(f"\n{'='*60}")
        print(f"PROCESSANDO DESIGN {target_idx}: {target_design}")
        print(f"{'='*60}")

        # 1. Coleta trojans conhecidos (de todos os outros arquivos)
        trojan_gates_list = []

        for i in prefix_list:
            if i == target_prefix:  # Pula o arquivo target
                continue

            result_file = f"/content/trojan_gabarito/result{i}.txt"
            print(f"Coletando trojans de: {result_file}")

            try:
                with open(result_file, "r") as f:
                    inside_block = False
                    for line in f:
                        line = line.strip()
                        if line == "TROJAN_GATES":
                            inside_block = True
                            continue
                        elif line == "END_TROJAN_GATES":
                            inside_block = False
                            continue
                        if inside_block and line.startswith("g"):
                            prefixed = f"{i}{line}"
                            trojan_gates_list.append(prefixed)
            except FileNotFoundError:
                print(f"Arquivo não encontrado: {result_file}")
                continue

        print(f"Total de trojans conhecidos coletados: {len(trojan_gates_list)}")

        # 2. Coleta gabarito do arquivo target (para comparação posterior)
        target_result = f"/content/trojan_gabarito/result{target_prefix}.txt"
        target_trojans_gabarito = []

        try:
            with open(target_result, "r") as f:
                inside_block = False
                for line in f:
                    line = line.strip()
                    if line == "TROJAN_GATES":
                        inside_block = True
                        continue
                    elif line == "END_TROJAN_GATES":
                        inside_block = False
                        continue
                    if inside_block and line.startswith("g"):
                        target_trojans_gabarito.append(line)
        except FileNotFoundError:
            print(f"Arquivo gabarito não encontrado: {target_result}")
            target_trojans_gabarito = []

        print(f"Trojans no gabarito do design target: {len(target_trojans_gabarito)}")

        # 3. Processa features de todos os arquivos
        features_g = Features_gates()
        features_v = Vetorial_space()

        print("\nProcessando features de todos os designs...")
        for key, design_file in enumerate(trojan_exemples):
            design_path = [f"/content/trojan_gabarito/{design_file}"]
            try:
                design = load_design(design_path, prim_file="iccadPrim.py")
                print(f"Carregando design {key}: {design_file}")

                # Inicializa as features para o design atual
                features = initialize_features(design)
                annotate_features(design, features)
                annotate_features_cells(design, features)

                # Extrai as features numéricas para o design atual
                numeric_features = extract_numeric_features(features)

                # Salva as features no dicionário
                features_g.features_gates[key] = numeric_features

            except Exception as e:
                print(f"Erro ao processar {design_file}: {e}")
                continue

        # 4. Mescla as informações de todos os arquivos no espaço vetorial
        for id_file, gates in features_g.features_gates.items():
            for gate_key, gate_f in gates.items():
                new_key = f"{id_file}{gate_key}"
                if new_key not in features_v.gates_vetorial:
                    features_v.gates_vetorial[new_key] = gate_f

        print(f"Total de gates no espaço vetorial: {len(features_v.gates_vetorial)}")

        # 5. Gera hiperarestas
        print("Gerando hiperarestas...")
        features_hyperedges, hyperedge_weights = features_hyperedg(features_v.gates_vetorial, size_k)
        print(f"Hiperarestas geradas: {len(features_hyperedges)}")

        # 6. Busca por trojans no design target
        target_design_prefix = f"{target_idx}g"
        print(f"\nBuscando trojans para design prefix: {target_design_prefix}")

        candidates = find_trojan(features_hyperedges, trojan_gates_list, min_trojan_threshold, target_design_prefix)

        print(f"\nRESULTADOS PARA DESIGN {target_idx}:")
        print(f"Candidatos encontrados: {len(candidates)}")
        print_candidates(candidates)

        # 7. Lista gates encontrados
        gates_found, total_gates = list_gates(candidates, target_design_prefix)

        print(f"\nGates trojan detectados: {total_gates}")
        print("Lista:", gates_found)

        # 8. Salva resultados
        results_dict[target_idx] = {
            'design_file': target_design,
            'target_prefix': target_design_prefix,
            'known_trojans_count': len(trojan_gates_list),
            'gabarito_trojans': target_trojans_gabarito,
            'gabarito_count': len(target_trojans_gabarito),
            'candidates_found': len(candidates),
            'detected_gates': gates_found,
            'detected_count': total_gates,
            'candidates_details': candidates
        }

        print(f"\nResumo para design {target_idx}:")
        print(f"- Trojans conhecidos usados: {len(trojan_gates_list)}")
        print(f"- Trojans no gabarito: {len(target_trojans_gabarito)}")
        print(f"- Gates detectados: {total_gates}")

    return results_dict




In [ ]:
# Teste da função automatizada
# Executa a detecção automática para todos os designs

print("Iniciando detecção automatizada de trojans para todos os designs...")
print("="*80)

# Parâmetros
size_k = 10
min_trojan_threshold = 5

# Executa a função automatizada
results = automated_trojan_detection(trojan_exemples, prefix_list, size_k, min_trojan_threshold)

print("\n" + "="*80)
print("RESUMO FINAL DE TODOS OS DESIGNS")
print("="*80)

for design_idx, result in results.items():
    print(f"\nDesign {design_idx} ({result['design_file']}):")
    print(f"  - Trojans no gabarito: {result['gabarito_count']}")
    print(f"  - Candidatos encontrados: {result['candidates_found']}")
    print(f"  - Gates detectados: {result['detected_count']}")
    print(f"  - Taxa de detecção: {result['detected_count']}/{result['gabarito_count']} = {(result['detected_count']/max(result['gabarito_count'],1)*100):.1f}%")

print(f"\nTotal de designs processados: {len(results)}")

Streaming output truncated to the last 5000 lines.
  Trojan Count: 5
  Found Trojans: ['14g3318', '14g2264', '9g2300', '14g2380', '14g4905']
  Design Gates: ['16g1281', '16g1281', '16g1754', '16g2486', '16g69', '16g2058']
  Hyperedge: ['16g1281', '16g1281', '9g2300', '16g1754', '16g2486', '16g69', '14g2264', '14g4905', '14g3318', '16g2058', '14g2380']
----------------------------------------
Edge Key: 16e1284
  count unknown: 3
  Trojan Count: 8
  Found Trojans: ['14g3704', '14g3560', '14g5485', '14g6815', '14g8', '14g6115', '9g2592', '14g4648']
  Design Gates: ['16g1284', '16g1696', '16g2434']
  Hyperedge: ['16g1284', '14g3704', '14g6815', '16g1696', '14g8', '14g5485', '14g6115', '14g4648', '16g2434', '9g2592', '14g3560']
----------------------------------------
Edge Key: 16e1291
  count unknown: 3
  Trojan Count: 8
  Found Trojans: ['14g228', '14g5583', '14g6501', '14g3332', '14g4694', '14g1643', '14g4401', '14g1010']
  Design Gates: ['16g1291', '16g2005', '16g1030']
  Hyperedge: ['1

In [ ]:
"""
desig_s = "6g"
candidates= find_trojan(features_hyperedges, trojan_gates_list, 5, desig_s )
print_candidates(candidates)
"""


'\ndesig_s = "6g"\ncandidates= find_trojan(features_hyperedges, trojan_gates_list, 5, desig_s )\nprint_candidates(candidates)\n'

In [ ]:
"""
gates, total = list_gates(candidates, desig_s)

print("Lista de gates encontrados:", gates)
print("Total de gates encontrados:", total)
"""

'\ngates, total = list_gates(candidates, desig_s)\n\nprint("Lista de gates encontrados:", gates)\nprint("Total de gates encontrados:", total)\n'

In [ ]:
"""
def find_hyperedges_with_cross_file_trojans(features_hyperedges, trojan_gates_list):
    results = []

    # transforma a lista de trojans em um set (busca mais rápida)
    trojan_set = set(trojan_gates_list)

    for edge_key, nodes in features_hyperedges.items():
        # interseção: quais dos nodes da hyperedge estão na lista de trojans
        found_trojans = list(set(nodes) & trojan_set)
        trojan_count = len(found_trojans)

        info = {
            "edge_key": edge_key,
            "total_nodes": len(nodes),
            "trojan_count": trojan_count,
            "found_trojans": found_trojans,
            "hyperedge": nodes
        }

        results.append(info)

        # imprime só se houver trojan(s) nessa hyperedge
        if trojan_count > 0:
            print(f"Edge Key: {edge_key}")
            print(f"  Total Nodes: {len(nodes)}")
            print(f"  Trojan Count: {trojan_count}")
            print(f"  Found Trojans: {found_trojans}")
            print(f"  Hyperedge: {nodes}")
            print("-" * 40)

    return results

srch = find_hyperedges_with_cross_file_trojans(features_hyperedges, trojan_gates_list)   """

'\ndef find_hyperedges_with_cross_file_trojans(features_hyperedges, trojan_gates_list):\n    results = []\n\n    # transforma a lista de trojans em um set (busca mais rápida)\n    trojan_set = set(trojan_gates_list)\n\n    for edge_key, nodes in features_hyperedges.items():\n        # interseção: quais dos nodes da hyperedge estão na lista de trojans\n        found_trojans = list(set(nodes) & trojan_set)\n        trojan_count = len(found_trojans)\n\n        info = {\n            "edge_key": edge_key,\n            "total_nodes": len(nodes),\n            "trojan_count": trojan_count,\n            "found_trojans": found_trojans,\n            "hyperedge": nodes\n        }\n\n        results.append(info)\n\n        # imprime só se houver trojan(s) nessa hyperedge\n        if trojan_count > 0:\n            print(f"Edge Key: {edge_key}")\n            print(f"  Total Nodes: {len(nodes)}")\n            print(f"  Trojan Count: {trojan_count}")\n            print(f"  Found Trojans: {found_trojans

In [ ]:
"""
def find_trojans_design19_and_cross_file_flexible(features_hyperedges, gates_unknown, trojan_gates_list, min_trojans=3, min_design19=1):


    results = []

    # Convert lists to sets for faster lookup
    design19_trojans = set(gates_unknown)
    cross_file_trojans = set(trojan_gates_list)

    for edge_key, nodes in features_hyperedges.items():
        # Find trojans from design19 in this hyperedge
        design19_found = list(set(nodes) & design19_trojans)

        # Find trojans from other files in this hyperedge
        cross_file_found = list(set(nodes) & cross_file_trojans)

        total_trojans = len(design19_found) + len(cross_file_found)

        # More flexible criteria: at least min_design19 from design19 and min_trojans total
        if len(design19_found) >= min_design19 and total_trojans >= min_trojans:
            hyperedge_info = {
                "edge_key": edge_key,
                "total_nodes": len(nodes),
                "design19_trojans": design19_found,
                "design19_count": len(design19_found),
                "cross_file_trojans": cross_file_found,
                "cross_file_count": len(cross_file_found),
                "total_trojans": total_trojans,
                "hyperedge": nodes
            }
            results.append(hyperedge_info)

    # Sort by total number of trojans (descending)
    results.sort(key=lambda x: x["total_trojans"], reverse=True)

    print(f"Found {len(results)} hyperedges with at least {min_design19} design19 trojan(s) and {min_trojans} total trojans")
    print("-" * 60)

    for info in results[:10]:  # Show top 10
        print(f"Edge Key: {info['edge_key']}")
        print(f"  Total Nodes: {info['total_nodes']}")
        print(f"  Design19 Trojans ({info['design19_count']}): {info['design19_trojans']}")
        print(f"  Cross-file Trojans ({info['cross_file_count']}): {info['cross_file_trojans']}")
        print(f"  Total Trojans: {info['total_trojans']}")
        print("-" * 40)

    return results

# Test with flexible parameters
flexible_results = find_trojans_design19_and_cross_file_flexible(features_hyperedges, gates_unknown, trojan_gates_list, min_trojans=3, min_design19=1)

"""

'\ndef find_trojans_design19_and_cross_file_flexible(features_hyperedges, gates_unknown, trojan_gates_list, min_trojans=3, min_design19=1):\n\n\n    results = []\n\n    # Convert lists to sets for faster lookup\n    design19_trojans = set(gates_unknown)\n    cross_file_trojans = set(trojan_gates_list)\n\n    for edge_key, nodes in features_hyperedges.items():\n        # Find trojans from design19 in this hyperedge\n        design19_found = list(set(nodes) & design19_trojans)\n\n        # Find trojans from other files in this hyperedge\n        cross_file_found = list(set(nodes) & cross_file_trojans)\n\n        total_trojans = len(design19_found) + len(cross_file_found)\n\n        # More flexible criteria: at least min_design19 from design19 and min_trojans total\n        if len(design19_found) >= min_design19 and total_trojans >= min_trojans:\n            hyperedge_info = {\n                "edge_key": edge_key,\n                "total_nodes": len(nodes),\n                "design19_tro

In [ ]:

def find_hyperedges_with_most_unique_trojans_and_design19(features_hyperedges, trojan_gates_list):
    trojan_set = set(trojan_gates_list)
    results = []

    for edge_key, nodes in features_hyperedges.items():
        # Trojans presentes na hyperedge
        found_trojans = [node for node in nodes if node in trojan_set]
        # Designs distintos dos trojans encontrados
        unique_designs = set(node.split("g")[0] for node in found_trojans)
        # Verifica se há gate do design 19
        has_design19 = any(node.startswith("19g") for node in nodes)

        if found_trojans and has_design19:
            info = {
                "edge_key": edge_key,
                "trojan_count": len(found_trojans),
                "unique_designs_count": len(unique_designs),
                "found_trojans": found_trojans,
                "hyperedge": nodes
            }
            results.append(info)

    # Ordena por maior número de trojans de designs diferentes
    results.sort(key=lambda x: x["unique_designs_count"], reverse=True)

    # Exibe os melhores resultados
    for info in results[:5]:  # mostra os top 5
        print(f"Edge Key: {info['edge_key']}")
        print(f"  Unique Trojan Designs: {info['unique_designs_count']}")
        print(f"  Trojan Count: {info['trojan_count']}")
        print(f"  Found Trojans: {info['found_trojans']}")
        print(f"  Hyperedge: {info['hyperedge']}")
        print("-" * 40)

    return results


'\ndef find_hyperedges_with_most_unique_trojans_and_design19(features_hyperedges, trojan_gates_list):\n    trojan_set = set(trojan_gates_list)\n    results = []\n\n    for edge_key, nodes in features_hyperedges.items():\n        # Trojans presentes na hyperedge\n        found_trojans = [node for node in nodes if node in trojan_set]\n        # Designs distintos dos trojans encontrados\n        unique_designs = set(node.split("g")[0] for node in found_trojans)\n        # Verifica se há gate do design 19\n        has_design19 = any(node.startswith("19g") for node in nodes)\n\n        if found_trojans and has_design19:\n            info = {\n                "edge_key": edge_key,\n                "trojan_count": len(found_trojans),\n                "unique_designs_count": len(unique_designs),\n                "found_trojans": found_trojans,\n                "hyperedge": nodes\n            }\n            results.append(info)\n\n    # Ordena por maior número de trojans de designs diferentes\n